In [1]:
import numpy as np
import pandas as pd
import math
from astropy.wcs import WCS,utils

In [2]:
unions_data = pd.read_csv('/scratch/merileo/unions.ugriz2.tsv', header=0, delim_whitespace=True)#, nrows=80000)

In [3]:
unions_data

,RA,Dec,CFIS_ID,CFIS_U_MAG_AUTO,CFIS_U_MAGERR_AUTO,HSC_G_MAG_AUTO,HSC_G_MAGERR_AUTO,CFIS_R_MAG_AUTO,CFIS_R_MAGERR_AUTO,PS_i_stk_kron,PS_i_stk_kron_err,PS_z_stk_kron,PS_z_stk_kron_err,CFIS_R_StarGal
0,0.050347,30.001614,2000240003934,-99.000,-99.000,-999.000,-999.000,22.385,0.038,22.714,0.094,22.217,0.151,1.946
1,0.036178,30.001557,2000240003938,-99.000,-99.000,-999.000,-999.000,23.259,0.109,22.154,0.097,21.421,0.111,2.184
2,0.109960,30.002372,2000240003943,-99.000,-99.000,-999.000,-999.000,23.802,0.131,-99.000,-99.000,-99.000,-99.000,3.416
3,0.016088,30.001573,2000240003945,-99.000,-99.000,-999.000,-999.000,25.132,0.267,-99.000,-99.000,-99.000,-99.000,1.255
4,0.003157,30.001583,2000240003947,-99.000,-99.000,-999.000,-999.000,25.116,0.148,-99.000,-99.000,-99.000,-99.000,0.326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39101129,358.315392,30.061453,3621240011325,23.882,0.191,-999.000,-999.000,22.591,0.040,22.599,0.095,22.137,0.064,1.734
39101130,358.333010,30.184642,3621240011332,24.423,0.200,23.873,0.209,22.997,0.053,22.219,0.071,22.958,0.123,1.315
39101131,358.329666,30.224158,3621240011333,23.891,0.225,-99.000,-99.000,23.640,0.139,-99.000,-99.000,-99.000,-99.000,1.494
39101132,358.322870,30.189825,3621240011337,21.445,0.032,20.392,0.017,19.455,0.004,19.217,0.005,18.946,0.004,2.185


In [7]:
# Get all the data that has simultaneous PS1-i, HSC-g, and CFIS-r photometry
unions_data = unions_data[(unions_data['PS_i_stk_kron']>0) &
            (unions_data['HSC_G_MAG_AUTO']>0) &
            (unions_data['CFIS_R_MAG_AUTO']>0) & (unions_data['CFIS_R_MAG_AUTO']<24.5)]

In [31]:
#useful info for CFIS (same for all of UNIONS)
res =  0.1857 #arcsecond per pixel
res *= 1/3600 #degrees per pixel
size = 10000  #pixels
deg_range = size/2 * res #range from central value

In [6]:
#useful functions
def tile_to_coord(tile, return_ra = False, return_dec = False):
    '''correspondance in degrees of ra,dec of tile name'''
    xxx,yyy = tile.split('.')
    dec  = int(yyy)/2 - 90
    ra = int(xxx)/(2*np.cos(math.radians(dec)))
    
    if return_ra is True:
        return ra
    if return_dec is True:
        return dec
    
def coord_to_tile(ra, dec, row):
    '''takes ra and dec coord (in degrees) and outputs the tile name centered there'''
    #convert to radians
    try: 
        yyy = int(round((2*(dec+90))))
        cosf = np.cos(np.deg2rad(yyy/2-90))
        xxx = int(round(ra*2*cosf))
        
        xxx = '{0:0=3d}'.format(xxx)
        yyy = '{0:0=3d}'.format(yyy)
        
        if xxx=='720':
            xxx='000'
            
        return xxx + '.' + yyy
    except e as error:
        print(error)
        print('ra and dec are not a number', row)
        return 0

In [8]:
# Add in tile information
unions_data['tile'] = unions_data.apply(lambda row: coord_to_tile(row['RA'], row['Dec'], row), axis=1)

In [9]:
unions_data

,RA,Dec,CFIS_ID,CFIS_U_MAG_AUTO,CFIS_U_MAGERR_AUTO,HSC_G_MAG_AUTO,HSC_G_MAGERR_AUTO,CFIS_R_MAG_AUTO,CFIS_R_MAGERR_AUTO,PS_i_stk_kron,PS_i_stk_kron_err,PS_z_stk_kron,PS_z_stk_kron_err,CFIS_R_StarGal,tile
2894,0.269950,30.101200,2000240016741,-99.000,-99.000,23.849,0.144,22.754,0.050,22.561,0.105,-99.000,-99.000,2.043,000.240
2946,0.287798,30.105164,2000240016810,-99.000,-99.000,22.654,0.067,21.451,0.012,20.718,0.019,20.318,0.025,0.980,000.240
3020,0.255747,30.111025,2000240016911,-99.000,-99.000,23.007,0.111,21.495,0.013,21.196,0.023,21.042,0.042,0.987,000.240
3151,0.254848,30.123517,2000240017087,-99.000,-99.000,22.311,0.057,20.923,0.009,20.154,0.010,19.748,0.017,0.998,000.240
3166,0.262637,30.125275,2000240017110,-99.000,-99.000,23.022,0.126,22.078,0.029,21.515,0.041,21.238,0.068,1.901,000.240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39101123,358.320669,30.206895,3621240011305,24.107,0.099,23.921,0.134,23.292,0.064,23.995,0.246,23.746,0.248,1.168,621.240
39101126,358.577186,30.245010,3621240011315,21.194,0.018,18.512,0.002,17.402,0.001,17.129,0.001,16.909,0.001,1.022,621.240
39101127,358.451673,30.207172,3621240011316,25.356,0.371,23.652,0.230,23.465,0.065,22.135,0.085,22.966,0.131,0.988,621.240
39101130,358.333010,30.184642,3621240011332,24.423,0.200,23.873,0.209,22.997,0.053,22.219,0.071,22.958,0.123,1.315,621.240


In [10]:
# Over 7 million sources!

In [34]:
# Save the new file
unions_data.to_csv("unions.ugriz2_matchingbands_PSiHSCgCFISr_fixed.tsv", sep='\t')